In [13]:
import sys
import os

# Add parent directory to sys.path
sys.path.append(os.path.abspath(".."))


In [14]:
from models.spin_system import SpinSystem

import pandas as pd
import numpy as np
import tensorflow as tf

import plotly.express as px
import matplotlib.pyplot as plt


In [15]:
def generate_periodic_nn_coupling_tensor(D, L):
    """
    Vectorized nearest-neighbor coupling tensor with periodic boundaries.
    J[i1,...,iD,j1,...,jD] = 1 if periodic Manhattan distance = 1, else 0
    """
    # Generate all coordinates: shape (N, D)
    coords = np.array(np.meshgrid(*[np.arange(L)]*D, indexing='ij')).reshape(D, -1).T
    N = coords.shape[0]

    # Compute pairwise differences with broadcasting
    diff = np.abs(coords[:, None, :] - coords[None, :, :])
    
    # Apply periodic boundary
    diff = np.minimum(diff, L - diff)

    # Manhattan distance
    manhattan_dist = diff.sum(axis=2)

    # Nearest neighbors mask
    nn_mask = (manhattan_dist == 1)

    # Create empty tensor and set neighbors
    J_tensor = np.zeros((L,)*D*2, dtype=np.float32)
    
    # Get indices where nn_mask is True
    idx_i, idx_j = np.nonzero(nn_mask)
    
    # Set values in the tensor
    for i, j in zip(idx_i, idx_j):
        J_tensor[tuple(coords[i]) + tuple(coords[j])] = 1.0

    return J_tensor


def run_temperature_sweep(D, L, betas, J=None, steps=None, dynamic_theta = True, spherical = True, ising = False,theta_max=0.1):
    results = []
    if steps == None:
        steps = L**D

    for beta in betas:
        
        system = SpinSystem(D, L, J=J, keep_history=True, spherical=spherical, ising=ising)
        if dynamic_theta == True and ising == False:
            theta_max = theta_max/beta
            print(f"Sweeping with beta {beta} with max rotation of {theta_max} radian")
        else: 
            print(f"Sweeping with beta {beta}")
        system.metropolis_sweep(beta, steps=steps, theta_max=theta_max)

        df = pd.DataFrame(system.history.get()).T
        df["beta"] = beta
        results.append(df)

    return pd.concat(results, ignore_index=True)

In [16]:
D = 2
L = 100
N = L ** D


J = generate_periodic_nn_coupling_tensor(D, L)

In [ ]:
betas = [0.1, 10, 100, 1000]
df = run_temperature_sweep(D, L, betas, J = J, steps=10000, dynamic_theta=False, spherical=False, ising=True)

Sweeping with beta 0.1
Metropolis sweep: 7509/10000 accepted (75.09%)
Sweeping with beta 10
Metropolis sweep: 4988/10000 accepted (49.88%)
Sweeping with beta 100
Metropolis sweep: 4936/10000 accepted (49.36%)
Sweeping with beta 1000
Metropolis sweep: 4853/10000 accepted (48.53%)


In [18]:
df['abs_magnetization'] = df['magnetization'].abs()

df_accepted = df[df['accepted'] == True].copy()

df_accepted['step'] = df_accepted.groupby('beta').cumcount()

fig = px.line(
    df_accepted,
    x='step',
    y='abs_magnetization',
    color='beta',
    labels={
        'step': 'Step',
        'abs_magnetization': 'Magnetization ⟨s⟩',
        'beta': 'β'
    },
    title='Magnetization evolution at different temperatures'
)

fig.update_layout(
    template='plotly_white',
    legend_title='β',
    width=1200,
    height=500
)

fig.show()